

# **Análise de Dados: COVID-19 Dashboard**


[Braian Fernandes](https://www.linkedin.com/in/braian-fernandes/)

---

# **COVID Dashboard**

## 1\. Contexto

**1.1 - TLDR**

  

*   Dashboard:
    
    - Googlie Data studio [(link)](https://lookerstudio.google.com/reporting/e0889574-ff2c-4af3-a0cc-de1ff3cd358a);

*   Processamento:

    - Kaggle Notebook [(link)](https://www.kaggle.com/braianalex);

*   Fontes:
    
    - Casos pela Universidade John Hopkins [(link)](https://systems.jhu.edu/research/public-health/ncov/);
    - Vacinação pela Universidade de Oxford [(link)](https://ourworldindata.org/).





**1.2 - Pandemia Coronavírus**

A COVID-19 é uma infecção respiratória aguda provocada pelo SARs-Cov-2, de elevada transmissibilidade e de distribuição global.

A disponibilidade de dados sobre a evolução da pandemia no tempo em uma determinada região geográfica é fundamental para o combate. Este projeto busca construir um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação no Brasil no ano de 2021.

**1.3 - Dados**

Os dados sobre casos da COVID-19 são compilados pelo centro de ciência de sistemas e engenharia da universidade americana John Hopkins. Os dados são atualizados diariamente desde janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados etc.). O website do projeto pode ser acessado no [link](https://systems.jhu.edu/research/public-health/ncov/). Já os dados estão disponíveis no [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports).

A seguir, estão descritos os dados derivados do seu processamento:

 • date: data de referência;

 • state: estado;

 • country: país;

 • population: população estimada;

 • confirmed: número acumulado de infectados;

 • confirmed_1d: número diário de infectados;

 • confirmed_moving_avg_7d: média móvel de 7 dias do número diário de infectados;

 • confirmed_moving_avg_7d_rate_14d: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;

 • deaths: número acumulado de mortos;

 • deaths_1d: número diário de mortos;

 • deaths_moving_avg_7d: média móvel de 7 dias do número diário de mortos;

 • deaths_moving_avg_7d: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;

 • month: mês de referência;

 • year: ano de referência.

Os dados sobre vacinação da COVID-19 são compilados pelo projeto Nosso Mundo em Dados (Our World in Data ou OWID) da universidade britânica de Oxford disponível no [link](https://www.ox.ac.uk/). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de países. O website do projeto pode ser acessado no [link](https://ourworldindata.org/). A seguir, estão descritos os dados derivados do seu processamento:

• date: data de referência;

• country: país;

• population: população estimada;

• total: número acumulado de doses administradas;

• one_shot: número acumulado de pessoas com uma dose;  

• one_shot_perc: número acumulado relativo de pessoas com uma dose;

• two_shots: número acumulado de pessoas com duas doses; two_shot_perc:

• número acumulado relativo de pessoas com duas doses;

• three_shots: número acumulado de pessoas com três doses;

• three_shot_perc: número acumulado relativo de pessoas com três doses;

• month: mês de referência;

• year: ano de referência.

## 2\. Pacotes e bibliotecas

Nesta seção, vamos baixar os pacotes Python responsáveis por ler os dados brutos e prepará-los para a visualização.

In [ ]:
# importando as bibliotecas:

import math
from typing import Iterator
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

## 3\. Extração

**Casos**

Os dados disponilizados, em relação aos casos de Covid-19, estão compilados em um arquivo por dia. Portando, primeiramente precisaremos iterar dentro do um intervalo de tempo para extraí-lo.

In [ ]:
# Vamos criar uma função para realizar essa iteração:

def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [ ]:
# Definindo o corte temporal:

start_date = datetime(2021,1, 1)
end_date = datetime(2021, 12, 31)

In [ ]:
# Agora vamos selecionar as colinas de interesse de forma interativa:

cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  casos = pd.read_csv(data_source_url, sep=',')

  casos = casos.drop([ 'FIPS', 'Admin2', 'Last_Update' , 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key' , 'Case_Fatality_Ratio'] , axis=1)
  casos = casos.query('Country_Region == "Brazil"').reset_index(drop=True)
  casos['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = casos
    cases_is_empty = False

  else:
    cases = pd.concat([cases, casos], ignore_index=True)

---

**Vacinação**

Vamos processar os dados de vacinação da universidade de Oxford.

Os dados estão compilados em um único arquivo:

In [ ]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

<ipython-input-5-608e6d2dd56d>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


In [ ]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN


Vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.

In [ ]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [ ]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-09


## 4\. Transformação

Vamos manipular os dados para a construção do Dahsboard. O intuito é garantir uma boa granularidade.

Vamos começar manipulando os dados sobre os casos de Covid-19:

In [ ]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [ ]:
cases.shape

(9828, 6)

In [ ]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


Os dados apresentados estão bem consolidades e correspondendo a coluna de interesse. Outro ponto positivo a se observar, é ausência de dados nulos.

Através disso, podemos partir para o enriquecimento dos dados, como renomear as colunas, ajustar os nomes dos Estados...

In [ ]:
# Vamos renomear algumas colunas. Isso não quer dizer que ela estivessem erradas, mas o motivo dessa renomeação é apenas para uma melhor representação.

cases = cases.rename(columns={ 'Province_State': 'state', 'Country_Region': 'country' })
for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [ ]:
# Devido a acentuação, vamos também redefinir o nome dos Estados:

states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo':
    'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}
cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

Vamos cria duas colunas através de chaves temporais, e uma através de população estimada, para que a gente possa ter a possibilidade de criar outros filtros e enriquecer ainda mais o nosso dado:

In [ ]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year'] = cases['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

Agora, vamos gerar o número de casos por dia, a média móvel(7 dias) e  também a estabilidade de casos(14 dias).

In [ ]:
cases_ = []

def get_trend(rate: float) -> str:
    if np.isnan(rate):
        return np.NaN

    if rate < 0.75:
        status = 'downward'
    elif rate > 1.15:
        status = 'upward'
    else:
        status = 'stable'

    return status

for state in cases['state'].drop_duplicates():
    print(f"Processando estado: {state}")

    cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
    cases_per_state = cases_per_state.sort_values(by=['date'])

    cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
    cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
    cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d'] / cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
    cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

    cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
    cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
    cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d'] / cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
    cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

    cases_.append(cases_per_state)

cases = pd.concat(cases_, ignore_index=True)

cases_ = None

Processando estado: Acre
Processando estado: Alagoas
Processando estado: Amapá
Processando estado: Amazonas
Processando estado: Bahia
Processando estado: Ceará
Processando estado: Distrito Federal
Processando estado: Espírito Santo
Processando estado: Goiás
Processando estado: Maranhao
Processando estado: Mato Grosso
Processando estado: Mato Grosso do Sul
Processando estado: Minas Gerais
Processando estado: Pará
Processando estado: Paraíba
Processando estado: Paraná
Processando estado: Pernambuco
Processando estado: Piauí
Processando estado: Rio Grande do Norte
Processando estado: Rio Grande do Sul
Processando estado: Rio de Janeiro
Processando estado: Rondônia
Processando estado: Roraima
Processando estado: Santa Catarina
Processando estado: São Paulo
Processando estado: Sergipe
Processando estado: Tocantins


Vamos garantir a consistencia dos dados fazendo o *type casting* das colunas:

In [ ]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

Vamos reordernar as colunas, colocando a coluna 'date' como primeira por ser a chave temporal.

In [ ]:
cases = cases[[ 'date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year' ]]

In [ ]:
cases.head(30)

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021
5,2021-01-06,Brazil,Acre,881935,42378,208,<NA>,NaN,NaN,814,6,<NA>,NaN,NaN,2021-01,2021
6,2021-01-07,Brazil,Acre,881935,42478,100,<NA>,NaN,NaN,821,7,<NA>,NaN,NaN,2021-01,2021
7,2021-01-08,Brazil,Acre,881935,42814,336,161,NaN,NaN,823,2,4,NaN,NaN,2021-01,2021
8,2021-01-09,Brazil,Acre,881935,42908,94,139,NaN,NaN,823,0,4,NaN,NaN,2021-01,2021
9,2021-01-10,Brazil,Acre,881935,43127,219,155,NaN,NaN,825,2,4,NaN,NaN,2021-01,2021


---

Agora, vamos manipular os dados referentes a vacinação:

In [ ]:
vaccines.tail()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
1669,Brazil,215313504,NaN,NaN,NaN,NaN,2024-07-31
1670,Brazil,215313504,NaN,NaN,NaN,NaN,2024-08-01
1671,Brazil,215313504,NaN,NaN,NaN,NaN,2024-08-02
1672,Brazil,215313504,NaN,NaN,NaN,NaN,2024-08-03
1673,Brazil,215313504,NaN,NaN,NaN,NaN,2024-08-04


In [ ]:
vaccines.shape

(1674, 7)

In [ ]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1674 non-null   object        
 1   population               1674 non-null   int64         
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1674 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 91.7+ KB


Vamos começar tratando os dados faltantes(nulos).

In [ ]:
vaccines = vaccines.fillna(method='ffill')

<ipython-input-25-274988f553eb>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vaccines = vaccines.fillna(method='ffill')


Vamos ajustar a base de dados para que o recorte temporal seja o mesmo para ambos as bases:

In [ ]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

Vamos alterar o nome das colunas para que sejam mais consistentes com os dados apresentados:

In [ ]:
vaccines = vaccines.rename(columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
})

Vamos também criar as colunas correspondentes as chaves temporais e outras correspondentes aos dados relativos, para termos uma noção em dados percentuais de quantas pessoas se vacinaram:

In [ ]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year'] = vaccines['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

Vamos garantir a consistência dos dados realizando o *type casting* das colunas:

In [ ]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

Vamos reorganizar as colunas para uma melhor representação dos dados:

In [ ]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [ ]:
vaccines.tail()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
360,2021-12-27,Brazil,215313504,329011365,165952037,0.7707,142764283,0.6631,25218893,0.1171,2021-12,2021
361,2021-12-28,Brazil,215313504,329861730,166062249,0.7713,142965728,0.6640,25758909,0.1196,2021-12,2021
362,2021-12-29,Brazil,215313504,330718457,166143380,0.7716,143282084,0.6655,26219623,0.1218,2021-12,2021
363,2021-12-30,Brazil,215313504,331164041,166185628,0.7718,143398692,0.6660,26507937,0.1231,2021-12,2021
364,2021-12-31,Brazil,215313504,331273910,166195505,0.7719,143436012,0.6662,26571077,0.1234,2021-12,2021


## 5\. Carregamento

Com os dados manipulados, vamos carregá-lo e fazer o download em um arquivo csv para iniciarmos a construção do Dashboard no Google Data studios.

In [ ]:
# Arquivo sobre os casos:

cases.to_csv('./covid_casos.csv', sep=',', index=False)

In [ ]:
# Arquivo sobre a vacinação:

vaccines.to_csv('./vacinacao_covid.csv', sep=',', index=False)